In [172]:
import mido
import numpy as np
import matplotlib.pyplot as plt
import Parser
import PreProcessor as pp


In [173]:
dataset = pp.load_dataset("adl-piano-midi")
Songs = pp.files_to_songs(dataset["Classical"])

channel_to_ind, ind_to_channel, note_to_ind, ind_to_note, velocity_to_ind, ind_to_velocity, tick_to_ind, ind_to_tick = pp.dicts_from_songs(Songs)

n_Channels = len(channel_to_ind)
n_Notes = len(note_to_ind)
n_Velocities = len(velocity_to_ind)
n_Ticks = len(tick_to_ind)

print("Number of channels  :",n_Channels,"\nNumber of notes     :",n_Notes,"\nNumber of velocities:",n_Velocities,"\nNumber of time ticks:",n_Ticks)

adl-piano-midi\adl-piano-midi\Classical\Classical\Alexander Borodin\Nocturne.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Antonio Salieri\Invention.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Bela Bartok\Andante.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Bela Bartok\Romanian Folk Dance.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Claude Debussy\Clair De Lune.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Franz Schubert\Impromptu in C-Moll.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Franz Schubert\Impromptu in Ges-dur.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Franz Schubert\Trois Marches militaires 3.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Frederic Chopin\Prelude Op 28 No7.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Giacomo Puccini\O Mio Babibino Caro.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Jean-Baptiste Lully\Minuet.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Johann Sebas

In [174]:
seq_length=5
n_batch=64

AllNotes = np.array([])
for song in Songs.values():
    AllNotes = np.append(AllNotes,song[:,1])
OneHotNotes = pp.one_hot_encode(note_to_ind,AllNotes)
X_train, y_train = pp.label_sequences(OneHotNotes,seq_length,n_batch)
print(X_train.shape)

(442560, 5, 105)


In [175]:
print(AllNotes)

[58. 62. 63. ... 52. 60. 64.]


In [47]:
from keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Softmax
from keras import optimizers
import tensorflow as tf

In [48]:
dim=len(note_to_ind)
print(dim)

105


In [49]:
def create_weight(notes):
    # Calculer les poids en fonction de la fréquence des notes
    note_counts = {}
    for element in notes:
        if element in note_counts:
            note_counts[element] += 1
        else:
            note_counts[element] = 1
    
    # Calculer les poids pour chaque note
    total_notes = sum(note_counts.values())
    weights = {note: count/total_notes for note, count in note_counts.items()}
    
    return weights

In [89]:
def create_model(dim,weights,seq_length,training_one_hot,label_one_hot):
    model = Sequential()
    model.add(SimpleRNN(units=dim, input_shape=(seq_length,dim), return_sequences=True))
    model.add(Dense(units=dim, activation='softmax'))
    model.summary()
    # Compiler le modèle
    opt = optimizers.Adagrad(learning_rate=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    # Train the model
    model.fit(training_one_hot, label_one_hot, epochs=20, batch_size=64)
    return model

In [112]:
model.weights

[<KerasVariable shape=(105, 105), dtype=float32, path=sequential_11/simple_rnn_9/simple_rnn_cell/kernel>,
 <KerasVariable shape=(105, 105), dtype=float32, path=sequential_11/simple_rnn_9/simple_rnn_cell/recurrent_kernel>,
 <KerasVariable shape=(105,), dtype=float32, path=sequential_11/simple_rnn_9/simple_rnn_cell/bias>,
 <KerasVariable shape=(105, 105), dtype=float32, path=sequential_11/dense_8/kernel>,
 <KerasVariable shape=(105,), dtype=float32, path=sequential_11/dense_8/bias>]

In [118]:
print(X_train.shape)

(442560, 5, 105)


In [162]:

def next_sequence(notes, model):
  
    input=notes #set de 5 notes 
    notes = np.expand_dims(notes, axis=0)
    prediction=model.predict(notes)
    #note=tf.squeeze(prediction,axis=-1)


    return prediction

In [149]:
print(X_train[0,:,:].shape)

(5, 105)


In [100]:
weight=create_weight(AllNotes)
print(weight)

{58.0: 0.02557318227727403, 62.0: 0.03794899372532893, 63.0: 0.026653230088595554, 67.0: 0.03840451179805365, 56.0: 0.023936842057694436, 60.0: 0.038751573186796305, 79.0: 0.018958228360067154, 84.0: 0.009170916436384506, 86.0: 0.006549879906817632, 91.0: 0.0026196808210170977, 96.0: 0.0010316942063795125, 74.0: 0.02885941980193098, 77.0: 0.019553836316268728, 82.0: 0.008943157400022144, 87.0: 0.004481068659859503, 72.0: 0.031385556733013534, 89.0: 0.003576359154309007, 59.0: 0.027092479658722968, 65.0: 0.03265856706125317, 75.0: 0.01915345039123489, 41.0: 0.006689065984594632, 46.0: 0.009760649655537053, 48.0: 0.015222347499734508, 55.0: 0.031884457479331094, 70.0: 0.023694622130134464, 44.0: 0.009082343477799142, 61.0: 0.02438874490761976, 68.0: 0.024320959480130962, 51.0: 0.016741644881183442, 29.0: 0.0011044505652174894, 57.0: 0.03283164585277457, 28.0: 0.0004817284380203944, 31.0: 0.0020972811265034244, 32.0: 0.0016765595732229485, 33.0: 0.0024836580631895757, 45.0: 0.011678977253

In [101]:
print(ind_to_note)

{0: 0, 1: 5, 2: 6, 3: 11, 4: 12, 5: 14, 6: 17, 7: 18, 8: 19, 9: 20, 10: 21, 11: 22, 12: 23, 13: 24, 14: 25, 15: 26, 16: 27, 17: 28, 18: 29, 19: 30, 20: 31, 21: 32, 22: 33, 23: 34, 24: 35, 25: 36, 26: 37, 27: 38, 28: 39, 29: 40, 30: 41, 31: 42, 32: 43, 33: 44, 34: 45, 35: 46, 36: 47, 37: 48, 38: 49, 39: 50, 40: 51, 41: 52, 42: 53, 43: 54, 44: 55, 45: 56, 46: 57, 47: 58, 48: 59, 49: 60, 50: 61, 51: 62, 52: 63, 53: 64, 54: 65, 55: 66, 56: 67, 57: 68, 58: 69, 59: 70, 60: 71, 61: 72, 62: 73, 63: 74, 64: 75, 65: 76, 66: 77, 67: 78, 68: 79, 69: 80, 70: 81, 71: 82, 72: 83, 73: 84, 74: 85, 75: 86, 76: 87, 77: 88, 78: 89, 79: 90, 80: 91, 81: 92, 82: 93, 83: 94, 84: 95, 85: 96, 86: 97, 87: 98, 88: 99, 89: 100, 90: 101, 91: 102, 92: 103, 93: 104, 94: 105, 95: 106, 96: 107, 97: 108, 98: 109, 99: 110, 100: 112, 101: 113, 102: 115, 103: 118, 104: 119}


In [84]:
print(X_train.shape)

(442560, 5, 105)


In [91]:

model = create_model(dim,weight,seq_length,X_train,y_train)

c:\Users\anhly\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_9 (SimpleRNN)        │ (None, 5, 105)         │        22,155 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 5, 105)         │        11,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,285 (130.02 KB)

 Trainable params: 33,285 (130.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 24s 5ms/step - accuracy: 0.0481 - loss: 4.0644
Epoch 2/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - accuracy: 0.1088 - loss: 3.6104
Epoch 3/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 23s 5ms/step - accuracy: 0.1231 - loss: 3.4767
Epoch 4/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - accuracy: 0.1308 - loss: 3.4212
Epoch 5/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - accuracy: 0.1352 - loss: 3.3914
Epoch 6/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - accuracy: 0.1382 - loss: 3.3717
Epoch 7/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 23s 5ms/step - accuracy: 0.1410 - loss: 3.3566
Epoch 8/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 24s 5ms/step - accuracy: 0.1428 - loss: 3.3469
Epoch 9/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.1439 - loss: 3.3403
Epoch 10/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 32s 7ms/step - accuracy: 0.1454 - loss: 3.3327
Epoch 11/20
4426/4426 ━━━━━━━━━━━━━━━━━━━━ 28s 6ms/step - accuracy: 0.1467 - loss: 3.3255
Epoch 12/20
4426/44

In [94]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_9 (SimpleRNN)        │ (None, 5, 105)         │        22,155 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 5, 105)         │        11,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,572 (260.05 KB)

 Trainable params: 33,285 (130.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 33,287 (130.03 KB)

In [ ]:
morceau=[X_train[0,:,:]]
for k in range(200):
    print(k)
    test_result=next_sequence(morceau[-1],model)
    
    morceau.append(np.reshape(test_result, (5, 105)))